In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

df = pd.read_parquet("../../data/transactions.parquet")
df.head()

,transaction_id,transaction_date,customer_id,product_id,product_name,category_0_name,category_0_id,category_1_name,category_1_id,brand_name,brand_id,unit_quantity,unit_spend,store_id
0,7108,2023-01-12 17:44:29,1,15,Spawn Figure,Toys,1,Action Figures,1,McFarlane Toys,3,2,55.98,6
1,7108,2023-01-12 17:44:29,1,1317,Gone Girl,Books,8,Mystery & Thrillers,53,Alfred A. Knopf,264,1,10.49,6
2,4553,2023-02-05 09:31:42,1,509,Ryzen 3 3300X,Electronics,3,Computer Components,21,AMD,102,3,360.00,4
3,4553,2023-02-05 09:31:42,1,735,Linden Wood Paneled Mirror,Home,5,Home Decor,30,Pottery Barn,147,1,599.00,4
4,4553,2023-02-05 09:31:42,1,1107,Pro-V Daily Moisture Renewal Conditioner,Beauty,7,Hair Care,45,Pantene,222,1,4.99,4


In [ ]:
print(f"Number of unique customers: {df['customer_id'].nunique()}")
print(f"Number of unique transactions: {df['transaction_id'].nunique()}")

Number of unique customers: 4250
Number of unique transactions: 25490


In [ ]:
import numpy as np

df["period"] = np.where(df["transaction_date"] < pd.to_datetime("2023-06-01"), "p1", "p2")

In [ ]:
import importlib
import ibis
from pyretailscience import contribution


importlib.reload(contribution)

conn = ibis.connect("duckdb://")
conn.create_table("my_table", ibis.memtable(df, name="my_table"))
t = conn.table("my_table")

contribution = contribution.ContributionAnalysis(
    df=t,
    p1_label="p1",
    p2_label="p2",
    period_col="period",
    group_cols=["category_0_name"],
)
contribution.df

,category_0_name,customers_p1,customers_p2,customers_diff,customers_pct_diff,customers_contrib,transactions_p1,transactions_p2,transactions_diff,transactions_pct_diff,...,expected_spend_per_customer_p2,expected_vs_actual_spend_diff,expected_vs_actual_transactions_diff,expected_vs_actual_customers_diff,expected_vs_actual_units_diff,expected_vs_actual_price_per_unit_diff,expected_vs_actual_transactions_per_customer_diff,expected_vs_actual_units_per_transaction_diff,expected_vs_actual_spend_per_transaction_diff,expected_vs_actual_spend_per_customer_diff
0,Beauty,2519,2455,-64,-0.025407,-9228.579462,4639,5412,773,0.166631,...,182.472389,-13791.968750,-36.538169,311.640351,-159.178882,-0.706143,-0.337574,-0.013544,-2.063783,-28.781147
1,Books,2474,2426,-48,-0.019402,-2314.060398,4607,5215,608,0.131973,...,61.758884,-3117.804688,-298.930454,352.455418,-611.566490,0.277967,-0.509552,0.010603,0.733422,-10.257647
2,Clothing,2496,2458,-38,-0.015224,-31864.435097,4482,5374,892,0.199018,...,1060.311974,98724.500000,200.276147,413.892504,206.710063,4.998312,-0.344713,-0.049374,2.758480,-138.377068
3,Electronics,2460,2439,-21,-0.008537,-71115.798887,4600,5337,737,0.160217,...,4497.156577,-39666.000000,-84.543634,459.080754,-467.955552,25.193428,-0.550073,-0.051687,18.584103,-862.740423
4,Grocery,2437,2471,34,0.013952,546.936367,4491,5364,873,0.194389,...,23.291273,652.226562,167.253725,670.292428,166.319827,0.004416,-0.715166,-0.038730,-0.130053,-6.054122
5,Home,2447,2441,-6,-0.002452,-8024.747494,4434,5296,862,0.194407,...,1827.134806,-43873.750000,163.971253,511.883598,267.505572,-10.146135,-0.490698,-0.021158,-29.549034,-401.128251
6,Movies,2463,2425,-38,-0.015428,-3072.735781,4507,5358,851,0.188817,...,101.541711,8578.843750,127.461975,388.032432,320.754308,0.237076,-0.358322,0.007687,0.660406,-12.710371
7,Music,2475,2458,-17,-0.006869,-156218.296160,4527,5422,895,0.197703,...,12498.251590,-438178.000000,193.854825,489.559066,291.063735,-81.273725,-0.450124,-0.030276,-249.059427,-2667.538815
8,Sports,2438,2437,-1,-0.000410,-2904.136088,4462,5301,839,0.188032,...,3886.082668,244686.500000,119.757730,524.843137,370.040777,0.733442,-0.534417,0.018974,13.758349,-736.519271
9,Toys,2467,2432,-35,-0.014187,-6504.730613,4527,5195,668,0.147559,...,241.833113,-5947.562500,-175.633943,404.099185,-390.374358,0.806394,-0.512269,0.002325,1.942301,-42.628343
